In [3]:
import pandas as pd
from pykrx import stock
from datetime import datetime, timedelta

# 종목 리스트 (20개 종목코드 예시)
tickers = [
    "005930", "000660", "035420", "051910", "005380", "068270", "017670", "028260", "105560", "207940",
    "096770", "066570", "034220", "036570", "012330", "010950", "032830", "011200", "003550", "086790"
]

# 오늘 날짜 및 1개월 전 날짜 계산
today = datetime.today().strftime("%Y%m%d")
one_month_ago = (datetime.today() - timedelta(days=30)).strftime("%Y%m%d")

# 결과 저장용 리스트
result = []

# 각 종목에 대해 순거래대금 변동 분석
for ticker in tickers:
    # 일별 거래 데이터 가져오기
    trading_data = stock.get_market_trading_value_by_date(one_month_ago, today, ticker, on="순매수")
    
    # 외국인 순거래대금 필터링
    foreign_net_value = trading_data["외국인합계"]
    
    # 양수(+)와 음수(-) 순매수 평균 계산
    positive_avg = foreign_net_value[foreign_net_value > 0].mean()
    negative_avg = foreign_net_value[foreign_net_value < 0].mean()
    
    # 평균 대비 ±200% 이상 변동한 날짜 필터링
    significant_dates = foreign_net_value[
        (foreign_net_value > positive_avg * 3) | (foreign_net_value < negative_avg * 2)
    ]
    
    # 변동이 발생한 날짜와 종목 코드 저장
    for date, value in significant_dates.items():
        result.append({
            "Ticker": ticker,
            "Date": date.strftime("%Y-%m-%d"),
            "Net Trading Value": value
        })

# 결과를 데이터프레임으로 변환
result_df = pd.DataFrame(result)

# 결과 출력
if not result_df.empty:
    print(result_df)
else:
    print("No significant changes in net trading value for the given period.")


    Ticker        Date  Net Trading Value
0   005930  2024-12-19      -327745823300
1   005930  2024-12-20      -249834853800
2   000660  2024-12-20      -188750884400
3   000660  2025-01-09       443736345500
4   035420  2024-12-11        59192681500
5   051910  2024-12-11       -13010224500
6   005380  2025-01-06       -43337565000
7   068270  2024-12-13        25026128700
8   068270  2025-01-02       -46325226500
9   017670  2024-12-20       -15414680700
10  017670  2025-01-09       -10293875400
11  207940  2024-12-17       -14862407000
12  207940  2025-01-02       -19677927000
13  207940  2025-01-07        48524965000
14  096770  2024-12-17        -5910257300
15  096770  2025-01-09        18117627000
16  066570  2024-12-18        19221588400
17  066570  2024-12-19       -11662605600
18  034220  2024-12-20        -2964367320
19  034220  2024-12-23         5298216030
20  036570  2024-12-27        -2872830000
21  012330  2024-12-13        26404536500
22  012330  2024-12-20       -1305

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from pykrx import stock
from datetime import datetime, timedelta

# 🧩 네이버 금융에서 종목코드 및 종목명 가져오기
def get_top_stocks():
    url = "https://finance.naver.com/sise/sise_market_sum.naver?sosok=0"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
    }

    response = requests.get(url, headers=headers)
    response.encoding = 'euc-kr'
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', class_='type_2')

    rows = table.find_all('tr')[2:]  # 첫 두 줄은 헤더이므로 제외
    data = []

    for row in rows:
        cols = row.find_all('td')
        if len(cols) > 1:
            name = cols[1].text.strip()
            code = cols[1].find('a')['href'].split('=')[-1]
            data.append((code, name))

    return dict(data)

# 🧩 종목코드와 종목명 가져오기
tickers = get_top_stocks()

# 📅 오늘 날짜 및 1개월 전 날짜 계산
today = datetime.today().strftime("%Y%m%d")
one_month_ago = (datetime.today() - timedelta(days=30)).strftime("%Y%m%d")

# 📋 결과 저장용 리스트
result = []

# 📈 각 종목에 대해 순거래대금 변동 분석
for ticker, name in tickers.items():
    try:
        # 일별 거래 데이터 가져오기
        trading_data = stock.get_market_trading_value_by_date(one_month_ago, today, ticker, on="순매수")

        # 외국인 순거래대금 필터링
        foreign_net_value = trading_data["외국인합계"]

        # 양수(+)와 음수(-) 순매수 평균 계산
        positive_avg = foreign_net_value[foreign_net_value > 0].mean()
        negative_avg = foreign_net_value[foreign_net_value < 0].mean()

        # 평균 대비 ±200% 이상 변동한 날짜 필터링
        significant_dates = foreign_net_value[
            (foreign_net_value > positive_avg * 4) | (foreign_net_value < negative_avg * 2.5)
        ]

        # 변동이 발생한 날짜와 종목 코드 저장
        for date, value in significant_dates.items():
            result.append({
                "Ticker": ticker,
                "Name": name,
                "Date": date.strftime("%Y-%m-%d"),
                "Net Trading Value": f"{int(value):,} 원"
            })

    except Exception as e:
        print(f"Error processing {name} ({ticker}): {e}")

# 📊 결과를 데이터프레임으로 변환 및 정렬
result_df = pd.DataFrame(result)

# 날짜 기준 오름차순 정렬
result_df = result_df.sort_values(by="Date", ascending=False).reset_index(drop=True)

# 📢 결과 출력
if not result_df.empty:
    print(result_df)
else:
    print("No significant changes in net trading value for the given period.")


In [ ]:
   Ticker       Name        Date   Net Trading Value
0   272210      한화시스템  2025-06-10    78,400,946,950 원
1   267250       HD현대  2025-06-10    19,697,326,100 원
2   323410      카카오뱅크  2025-06-10    -7,390,959,700 원
3   035720        카카오  2025-06-10   -71,038,649,050 원
4   012330      현대모비스  2025-06-09    25,608,168,000 원
5   042660       한화오션  2025-06-09   -56,047,224,200 원
6   352820        하이브  2025-06-09    27,010,707,250 원
7   005380        현대차  2025-06-09   101,372,680,250 원
8   009540   HD한국조선해양  2025-06-05   -22,703,556,750 원
9   402340      SK스퀘어  2025-06-05   -14,558,262,750 원
10  011200        HMM  2025-06-05   -19,746,311,925 원
11  316140     우리금융지주  2025-06-05    53,945,424,625 원
12  035420      NAVER  2025-06-04   -25,636,230,450 원
13  034730         SK  2025-06-04   -10,121,531,750 원
14  010140      삼성중공업  2025-06-02   -31,384,848,455 원
15  272210      한화시스템  2025-06-02   -29,552,776,950 원
16  003550         LG  2025-05-30   -10,302,400,550 원
17  006800     미래에셋증권  2025-05-30   -18,223,597,335 원
18  138040    메리츠금융지주  2025-05-30   -30,848,735,050 원
19  000270         기아  2025-05-30   -44,772,324,850 원
20  012450  한화에어로스페이스  2025-05-30  -153,910,703,500 원
21  034730         SK  2025-05-29   -13,477,836,750 원
22  035420      NAVER  2025-05-27   -34,586,977,400 원
23  066570       LG전자  2025-05-27   -19,389,281,900 원
24  005930       삼성전자  2025-05-27  -269,834,988,600 원
25  055550       신한지주  2025-05-26   -10,335,945,650 원
26  005380        현대차  2025-05-22   -76,966,859,900 원
27  032830       삼성생명  2025-05-22   -14,856,619,500 원
28  000660     SK하이닉스  2025-05-21   -22,354,723,100 원
29  207940   삼성바이오로직스  2025-05-21    54,314,821,500 원
30  086790     하나금융지주  2025-05-21   -11,076,673,550 원
31  018260    삼성에스디에스  2025-05-20    -3,861,181,500 원
32  051910       LG화학  2025-05-16   -31,561,275,900 원
33  373220   LG에너지솔루션  2025-05-16   -52,933,022,000 원
34  003550         LG  2025-05-14     7,609,393,050 원
35  010130       고려아연  2025-05-12    -7,064,019,000 원
36  068270       셀트리온  2025-05-12   -83,459,380,900 원
37  030200         KT  2025-05-12    -1,222,647,400 원
38  064350       현대로템  2025-05-12   -82,017,042,250 원
39  207940   삼성바이오로직스  2025-05-12   -65,085,155,000 원